In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Convert Labelme JSON (rectangle + points) to Ultralytics YOLO pose txt.
Assumptions:
- class "swd" -> id 0 (modify CLASS_TO_ID if you add more classes)
- keypoints labels: 'h' (body), 'lp' (left dot), 'rp' (right dot)
- if a kpt missing: use (0,0,0)
"""

from pathlib import Path
import json, argparse

CLASS_TO_ID = { "swd": 0, "p": 1}   # add more classes if needed
KPT_ORDER = ["h", "lp", "rp"]   # -> [body, left, right]

def norm(val, denom): return float(val) / float(denom)

def rect_from_points(pts):
    xs = [p[0] for p in pts]; ys = [p[1] for p in pts]
    x1, x2 = min(xs), max(xs); y1, y2 = min(ys), max(ys)
    return x1, y1, x2, y2

def inside(px, py, xyxy):
    x1, y1, x2, y2 = xyxy
    return (x1 - 1e-6) <= px <= (x2 + 1e-6) and (y1 - 1e-6) <= py <= (y2 + 1e-6)

def convert_one(json_path: Path, out_dir: Path):
    data = json.loads(Path(json_path).read_text())
    W, H = data["imageWidth"], data["imageHeight"]

    # collect shapes
    rects = []
    points = []
    for sh in data["shapes"]:
        t = sh.get("shape_type", "")
        lbl = sh.get("label", "")
        pts = sh.get("points", [])
        if t == "rectangle" and lbl in CLASS_TO_ID:
            rects.append({"label": lbl, "xyxy": rect_from_points(pts)})
        elif t == "point" and lbl in {"h", "lp", "rp"}:
            x, y = pts[0]
            points.append({"label": lbl, "pt": (x, y)})

    # if no rectangles, skip
    if not rects:
        return

    # simple association: assign each point to the first rect that contains it
    for r in rects:
        r["kpts"] = {k: None for k in KPT_ORDER}
        x1, y1, x2, y2 = r["xyxy"]
        for p in points:
            if inside(p["pt"][0], p["pt"][1], (x1, y1, x2, y2)):
                if p["label"] in r["kpts"] and r["kpts"][p["label"]] is None:
                    r["kpts"][p["label"]] = p["pt"]

    # write yolo txt
    out_dir.mkdir(parents=True, exist_ok=True)
    stem = Path(data["imagePath"]).stem
    out_txt = out_dir / f"{stem}.txt"

    lines = []
    for r in rects:
        x1, y1, x2, y2 = r["xyxy"]
        cx = (x1 + x2) / 2.0; cy = (y1 + y2) / 2.0
        w = (x2 - x1); h = (y2 - y1)
        cxn, cyn, wn, hn = norm(cx, W), norm(cy, H), norm(w, W), norm(h, H)

        cls_id = CLASS_TO_ID[r["label"]]
        parts = [str(cls_id),
                 f"{cxn:.6f}", f"{cyn:.6f}", f"{wn:.6f}", f"{hn:.6f}"]

        # kpts in KPT_ORDER = ["h","lp","rp"]
        for k in KPT_ORDER:
            if r["kpts"][k] is None:
                parts += ["0.000000", "0.000000", "0"]   # (x,y,v=0)
            else:
                x, y = r["kpts"][k]
                parts += [f"{norm(x,W):.6f}", f"{norm(y,H):.6f}", "2"]  # visible

        lines.append(" ".join(parts))

    out_txt.write_text("\n".join(lines))

if __name__ == "__main__":
    ann_dir = Path("/workspace/models/runs_yolov11_pose/cropped_objects_v5/swd");
    out_dir = Path("cropped_objects")
    for jp in ann_dir.glob("*.json"):
        try:
            convert_one(jp, out_dir)
        except Exception as e:
            print(f"[WARN] fail on {jp.name}: {e}")
